In [1]:
import torch
import os
import json
import pandas as pd
import pandas as pd
from pathlib import Path
from huggingface_hub import snapshot_download
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY_BRANDO")
#notebook_login()

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path
# Define the path to save the model
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)
# Download the model
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
#snapshot_download(repo_id=model_name,
#                  allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"],
#                  local_dir=mistral_models_path)

In [4]:
df_pacientes_reg = pd.read_csv(r'..\data\tratados\t_pacientes_reg.csv', sep=';', encoding='utf-8')
df_resultados = pd.DataFrame({
    'npac': df_pacientes_reg['npac'],
    'resultado': [None] * len(df_pacientes_reg)  # Adiciona uma nova coluna vazia
})

# Carrega os dados
df_cond_add = pd.read_csv(r'..\data\entrada_modelo\df_cond_add.csv', sep=';', encoding='utf-8') # Coluna coment_cond_add
df_cond_familia = pd.read_csv(r'..\data\entrada_modelo\df_cond_familia.csv', sep=';', encoding='utf-8') # Coluna coment_cond_familia
df_cons_reg = pd.read_csv(r'..\data\entrada_modelo\df_cons_reg.csv', sep=';', encoding='utf-8') # Coluna coment_cons_reg
df_diag_status = pd.read_csv(r'..\data\entrada_modelo\df_diag_status.csv', sep=';', encoding='utf-8') # Coluna coment_diag_status
df_gravidez = pd.read_csv(r'..\data\entrada_modelo\df_gravidez.csv', sep=';', encoding='utf-8') # Coluna coment_gravidez
df_lab_aac = pd.read_csv(r'..\data\entrada_modelo\df_lab_aac.csv', sep=';', encoding='utf-8') # Coluna coment_lab_aac
df_lab_basic = pd.read_csv(r'..\data\entrada_modelo\df_lab_basic.csv', sep=';', encoding='utf-8') # Coluna coment_lab_basic
df_lab_hemat = pd.read_csv(r'..\data\entrada_modelo\df_lab_hemat.csv', sep=';', encoding='utf-8') # Coluna coment_lab_hemat
df_lab_quimica = pd.read_csv(r'..\data\entrada_modelo\df_lab_quimica.csv', sep=';', encoding='utf-8') # Coluna coment_lab_quimica
df_lab_sorol = pd.read_csv(r'..\data\entrada_modelo\df_lab_sorol.csv', sep=';', encoding='utf-8') # Coluna coment_lab_sorol
df_liquor = pd.read_csv(r'..\data\entrada_modelo\df_liquor.csv', sep=';', encoding='utf-8') # Coluna coment_liquor
df_mri = pd.read_csv(r'..\data\entrada_modelo\df_mri.csv', sep=';', encoding='utf-8') # Coluna coment_mri
df_pot_evoc = pd.read_csv(r'..\data\entrada_modelo\df_pot_evoc.csv', sep=';', encoding='utf-8') # Coluna coment_pot_evoc
df_surtos_reg = pd.read_csv(r'..\data\entrada_modelo\df_surtos_reg.csv', sep=';', encoding='utf-8') # Coluna coment_surtos_reg
df_trat_dmd = pd.read_csv(r'..\data\entrada_modelo\df_trat_dmd.csv', sep=';', encoding='utf-8') # Coluna coment_trat_dmd
df_trat_odr = pd.read_csv(r'..\data\entrada_modelo\df_trat_odr.csv', sep=';', encoding='utf-8') # Coluna coment_trat_odr
df_trat_out = pd.read_csv(r'..\data\entrada_modelo\df_trat_out.csv', sep=';', encoding='utf-8') # Coluna coment_trat_out
df_vacinas_reg = pd.read_csv(r'..\data\entrada_modelo\df_vacinas_reg.csv', sep=';', encoding='utf-8') # Coluna coment_vacinas_reg

df_list=[df_pacientes_reg, df_cond_add, df_cond_familia, df_cons_reg, df_diag_status, df_gravidez, df_lab_aac, df_lab_basic, df_lab_hemat, df_lab_quimica, df_lab_sorol, df_liquor, df_mri, df_pot_evoc, df_surtos_reg, df_trat_dmd, df_trat_odr, df_trat_out, df_vacinas_reg]

# Carrega o json
with open(r'..\data\entrada_modelo\form.json', 'r') as f:
    dict_tabela = json.load(f)

In [5]:
def extrair_informacao_react_(id_paciente, temp_tab, df_list):
    for i in df_list:
        size=1
        temp_df = i[i['npac'] == id_paciente]
        if temp_df.columns[-1] == temp_tab['df_correspondente']:
            temp_df = i[i['npac'] == id_paciente]
            size = len(temp_df)
            break

    for i in df_list: # para cada Dataframe
        temp_df = i[i['npac'] == id_paciente]
        passos = []

        if temp_df.empty:
            continue

        for _ in range(size):
            passos.append(f"extraia e classifique as informações contidas no texto")

            # Adiciona um novo passo para cada chave do dicionário i
            for chave in temp_tab.keys():
                passos.append(f"Extraia a informação para o campo '{chave}': {temp_tab[chave]}\nTexto: {temp_df[temp_df.columns[-1]].to_string(index=False)}")
    print(passos)
    # TODO: Finaliza a implementação

    return None

def exrair_informacao_react(id_paciente, df_list, dict_tabela):
    respostas = {}
    
    for i in dict_tabela: # para cada tabela
        temp_tab = dict_tabela[i]
        respostas[temp_tab['titulo']] = {}
        temp_resp = extrair_informacao_react_(id_paciente, temp_tab, df_list)
        respostas[temp_tab['titulo']] = temp_resp # temp_resp é uma lista de dicionarios

    return respostas

# df_resultados
temp = exrair_informacao_react(4162, df_list, dict_tabela)

['extraia e classifique as informações contidas no texto', "Extraia a informação para o campo 'titulo': Registro de Pacientes\nTexto: []", "Extraia a informação para o campo 'df_correspondente': pais_reside\nTexto: []", "Extraia a informação para o campo 'npac': (contole interno)\nTexto: []", "Extraia a informação para o campo 'center_abrev': Centro de origem do paciente (interno)\nTexto: []", "Extraia a informação para o campo 'dt_pacientes_reg': Data de inclusão no BRANDO (*) [date]\nTexto: []", "Extraia a informação para o campo 'nome': Nome ou iniciais (*) [input text]\nTexto: []", "Extraia a informação para o campo 'n_reg_centro': Número interno do paciente no serviço/centro [input text/number]\nTexto: []", "Extraia a informação para o campo 'prontuario': Prontuário do paciente no serviço/centro [input text/number]\nTexto: []", "Extraia a informação para o campo 'convenio': Convênio [options]\nTexto: []", "Extraia a informação para o campo 'convenio_out_descr': Descrição [input te